In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import losses

import numpy as np
from keras import datasets  # mnist
from keras.utils import np_utils

import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance

In [ ]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import csv

# EDA

In [ ]:
train = pd.read_csv("C:\\Users\\munan\\Desktop\\dacon\\dku_ai\\train.csv", index_col=0)
test = pd.read_csv("C:\\Users\\munan\\Desktop\\dacon\\dku_ai\\test.csv", index_col=0)
submission = pd.read_csv("C:\\Users\\munan\\Desktop\\dacon\\dku_ai\\sample_submission.csv")

In [ ]:
train.isna().sum()

In [ ]:
train_shape = 199991
for col in train.columns[1:-1]:
    train = train.loc[np.logical_and(train[col]>test[col].min(), train[col]<test[col].max())]
    
print('최종 행 개수 :', train.shape[0])

In [ ]:
## 컬럼 위치 변경 
train2 = train[['class_train','redshift','nObserve','nDetect','u','g','r','i','z','dered_u','dered_g','dered_r','dered_i','dered_z','airmass_u','airmass_g','airmass_r','airmass_i','airmass_z'
]]
train = train[['class_train','redshift','nObserve','nDetect','u','g','r','i','z','dered_u','dered_g','dered_r','dered_i','dered_z','airmass_u','airmass_g','airmass_r','airmass_i','airmass_z'
]]
## 컬럼 위치 변경
test2 =test[['redshift','nObserve','nDetect','u','g','r','i','z','dered_u','dered_g','dered_r','dered_i','dered_z','airmass_u','airmass_g','airmass_r','airmass_i','airmass_z'
]]
# 컬럼 위치 변경
test = test[['redshift','nObserve','nDetect','u','g','r','i','z','dered_u','dered_g','dered_r','dered_i','dered_z','airmass_u','airmass_g','airmass_r','airmass_i','airmass_z'
]]

# 변수 생성

In [ ]:
## 변수간 add, diff (per 5point)
for i in range(1,6):
    for j in range(4,19-i):
        train2['diff'+str(i)+'_'+str(j)] = train2[train2.columns[j+i]] - train2[train2.columns[j]]
        test2['diff'+str(i)+'_'+str(j)] = test2[test2.columns[j-1+i]] - test2[test2.columns[j-1]]
        print('diff'+str(i)+'_'+str(j), ' : ', train2.columns[j+i], '-', train2.columns[j])
        print('diff'+str(i)+'_'+str(j), ' : ', test2.columns[j-1+i], '-', test2.columns[j-1])
    print('\n')

In [ ]:
for i in range(1,6):
    for j in range(4,19-i):
        train2['add_'+str(i)+'_'+str(j)] = train2[train2.columns[j+i]] + train2[train2.columns[j]]
        test2['add'+str(i)+'_'+str(j)] = test2[test2.columns[j-1+i]] + test2[test2.columns[j-1]]
        print('add'+str(i)+'_'+str(j), ' : ', train2.columns[j+i], '-', train2.columns[j])
        print('add'+str(i)+'_'+str(j), ' : ', test2.columns[j-1+i], '-', test2.columns[j-1])
    print('\n')

In [ ]:
## 파장 변수 별 파장 차이 per 5 point
diff_col = []
for col in ['u','g','r','i','z']:
    for i in range(3):
        diff_col.append(col + '_' + str(i))
mag_wave_diff_tr = pd.DataFrame(np.zeros((train2.shape[0], 15)), index=train2.index)
mag_wave_diff_tt = pd.DataFrame(np.zeros((test2.shape[0], 15)))

for i in range(0,15,5):
    for j in range(5):
        mag_wave_diff_tr.loc[:,j+i] = train2[train2.columns[4+j]] - train2[train2.columns[9+j+i]]
        mag_wave_diff_tt.loc[:,j+i] = test2[test2.columns[3+j]] - test2[test2.columns[8+j+i]]
        print(train.columns[4+j], ' - ',train2.columns[9+j+i], i+j)
        print(test.columns[3+j], ' - ',test2.columns[8+j+i], i+j)

In [ ]:
## 변수간 차이 
diff_feature = []
for c1, c2 in combinations(dered_col[::-1]+airmass_col[::-1]+z_col[::-1]+redshift_col[::-1],2):
    new_c = f'{c1}_{c2}_diff'
    train2[new_c] = train2[c1]-train2[c2]
    test2[new_c] = test2[c1]-test2[c2]
    diff_feature.append(new_c)

In [ ]:
## 변수의 개수 정보 활용
train2['t'] = 1
test2['t'] = 0
types = train2['class_train']
all_df = pd.concat([train2.drop('class_train', axis=1), test2], axis=0)
all_df['count_nDetect'] = all_df['nDetect'].map(all_df['nDetect'].value_counts())

train2 = all_df[all_df['t']==1].drop('t', axis=1)
test2 = all_df[all_df['t']==0].drop('t', axis=1)

train2['class_train']= types.values

In [ ]:
train2['t'] = 1
test2['t'] = 0
types = train2['class_train']
all_df = pd.concat([train2.drop('class_train', axis=1), test2], axis=0)
all_df['count_nObserve'] = all_df['nObserve'].map(all_df['nObserve'].value_counts())

train2 = all_df[all_df['t']==1].drop('t', axis=1)
test2 = all_df[all_df['t']==0].drop('t', axis=1)

train2['class_train']= types.values

In [ ]:
train2['t'] = 1
test2['t'] = 0
types = train2['class_train']
all_df = pd.concat([train2.drop('class_train', axis=1), test2], axis=0)
all_df['count_dered_u'] = all_df['dered_u'].map(all_df['dered_u'].value_counts())

train2 = all_df[all_df['t']==1].drop('t', axis=1)
test2 = all_df[all_df['t']==0].drop('t', axis=1)

train2['class_train']= types.values

In [ ]:
train2['t'] = 1
test2['t'] = 0
types = train2['class_train']
all_df = pd.concat([train2.drop('class_train', axis=1), test2], axis=0)
all_df['count_dered_z'] = all_df['dered_z'].map(all_df['dered_z'].value_counts())

train2 = all_df[all_df['t']==1].drop('t', axis=1)
test2 = all_df[all_df['t']==0].drop('t', axis=1)

train2['class_train']= types.values

In [ ]:
## 변수간 max-max, min-min, sum-sum 설정을 위한 컬럼 리스트
dered_col = [c for c in train.columns if c.find('dered')!=-1]
airmass_col = [c for c in train.columns if c.find('airmass')!=-1]
z_col = list(train.columns[4:9])
redshift_col = [c for c in train.columns if c.find('redshift')!=-1]

In [ ]:
for prefix, g in zip(['dered','airmass','z_','redshift'], [dered_col,airmass_col,z_col,redshift_col]):
    train2[f'{prefix}_max'] = train[g].max(axis=1)
    test2[f'{prefix}_max'] = test[g].max(axis=1)
    
    train2[f'{prefix}_min'] = train[g].min(axis=1)
    test2[f'{prefix}_min'] = test[g].min(axis=1)

    train2[f'{prefix}_std'] = train[g].std(axis=1)
    test2[f'{prefix}_std'] = test[g].std(axis=1)
    
    train2[f'{prefix}_sum'] = train[g].sum(axis=1)
    test2[f'{prefix}_sum'] = test[g].sum(axis=1)

In [ ]:
## test,train 변수 중간 체크
tmp = [x for x in train2.columns if x not in test2.columns]
tmp

In [ ]:
## 변수간 add, diff, div, mul combination 
a = train['class_train'].unique()
b = ['u','g','i','z','redshift','dered_u','dered_g','dered_r','dered_i','dered_z','nObserve']

train["nObserve"] = train["nObserve"].apply(np.log1p)
test["nObserve"] = test["nObserve"].apply(np.log1p)

for i , name in enumerate(b):
    for i2 , name2 in enumerate(b):
        if name==name2 :
            pass
        else :
            train[str(name)+"-"+str(name2)] = train[name]-train[name2]
            test[str(name)+"-"+str(name2)] = test[name]-test[name2]
            
for i , name in enumerate(b):
    for i2 , name2 in enumerate(b):
        if name==name2 :
            pass
        else :
            train[str(name)+"*"+str(name2)+"-"+str(name2)] = train[name]*train[name2]-train[name2]
            test[str(name)+"*"+str(name2)+"-"+str(name2)] = test[name]*test[name2]-test[name2]

for i , name in enumerate(b):
    for i2 , name2 in enumerate(b):
        if name==name2 :
            pass
        else :
            train[str(name)+"/"+str(name2)] = train[name]/train[name2]
            test[str(name)+"/"+str(name2)] = test[name]/test[name2]

for i , name in enumerate(b):
    for i2 , name2 in enumerate(b):
        if name==name2 :
            pass
        else :
            train[str(name)+"+"+str(name2)] = train[name]+train[name2]
            test[str(name)+"+"+str(name2)] = test[name]+test[name2]

train['r+nObserve'] = train['r'] + train['nObserve']
train['redshift+nObserve'] = train['redshift'] + train['nObserve']
train['u*airmass_u'] = train['u'] * train['airmass_u']
train['g*nDetect'] = train['g'] * train['nDetect']
train['i*nDetect'] = train['i'] * train['nDetect']
train['redshift*nDetect'] = train['redshift'] * train['nDetect']

train['z-dered_u2'] = train['z'] - train['dered_u']
train['z-dered_u3'] = train['z'] - train['dered_u']
train['z-dered_u4'] = train['z'] - train['dered_u']
train['z-dered_u5'] = train['z'] - train['dered_u']
train['z-dered_u6'] = train['z'] - train['dered_u']
train['z-dered_u7'] = train['z'] - train['dered_u']


test['r+nObserve'] = test['r'] + test['nObserve']
test['redshift+nObserve'] = test['redshift'] + test['nObserve']
test['u*airmass_u'] = test['u'] * test['airmass_u']
test['g*nDetect'] = test['g'] * test['nDetect']
test['i*nDetect'] = test['i'] * test['nDetect']
test['redshift*nDetect'] = test['redshift'] * test['nDetect']

test['z-dered_u2'] = test['z'] - test['dered_u']
test['z-dered_u3'] = test['z'] - test['dered_u']
test['z-dered_u4'] = test['z'] - test['dered_u']
test['z-dered_u5'] = test['z'] - test['dered_u']
test['z-dered_u6'] = test['z'] - test['dered_u']
test['z-dered_u7'] = test['z'] - test['dered_u']

In [ ]:
train2_ = pd.concat([train2 , train[train.columns[19:]]],axis =1)
test2_ = pd.concat([test2, test[test.columns[18:]]],axis =1)

# 하이퍼 파라미터 튜닝 BayesianOpt

## lgbm

In [ ]:
def lgbm_cv(learning_rate, n_estimators, num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                n_estimators=int(n_estimators),
                                num_leaves = int(num_leaves),
                                feature_fraction = feature_fraction,
                                bagging_fraction = feature_fraction,
                                max_depth = int(max_depth),
                                lambda_l1 = lambda_l1,
                                lambda_l2 = lambda_l2,
                                min_split_gain = min_split_gain,
                                min_child_weight = min_child_weight)
    RMSE = cross_val_score(model, new_train_set_under_454, Y, scoring='accuracy', cv=5).mean()
    return -RMSE

In [ ]:
pbounds = {'learning_rate' : (0.01, 0.3),
           'n_estimators' : (50, 1000),
           'num_leaves': (24, 45),
           'feature_fraction': (0.1, 0.9),
           'bagging_fraction': (0.8, 1),
           'max_depth': (5, 50),
           'lambda_l1': (0, 5),
           'lambda_l2': (0, 3),
           'min_split_gain': (0.001, 0.1),
           'min_child_weight': (5, 50)
          }

In [ ]:
lgbmBO = BayesianOptimization(f = lgbm_cv,pbounds = pbounds, verbose = 2, random_state = 1 )

In [ ]:
lgbmBO.maximize(init_points=4, n_iter = 16, acq='ei', xi=0.01)

In [ ]:
params=lgbmBO.max

## xgb

In [ ]:
def XGB_cv(max_depth,learning_rate, n_estimators, gamma,reg_alpha,reg_lambda,subsample,colsample_bytree, silent=True, nthread=-1):
    model = xgb.XGBClassifier(max_depth=int(max_depth),
                              learning_rate=learning_rate,
                              n_estimators=int(n_estimators),
                              silent=silent,
                              nthread=nthread,
                              gamma=gamma,
                            reg_alpha = reg_alpha,
                              reg_lambda = reg_lambda,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree)
    RMSE = cross_val_score(model, train2_set, y_train, scoring='accuracy', cv=5).mean()
    return -RMSE

In [ ]:

# 입력값의 탐색 대상 구간
pbounds = {'max_depth': (5, 10),
          'learning_rate': (0, 0.4),
          'n_estimators': (50, 1000),
          'gamma': (1., 0.01),
          'reg_alpha': (0, 10),
          'reg_lambda': (0, 10),
          'subsample': (0.7, 0.8),
          'colsample_bytree' :(0.5, 0.99)
          }

In [ ]:
xgboostBO = BayesianOptimization(f = XGB_cv,pbounds = pbounds, verbose = 2, random_state = 1 )

In [ ]:
# 반복적으로 베이지안 최적화 수행
# 시간을 줄이기 위해 N-n 값 10으로 지정
# acq='ei'사용
# xi=0.01 로 exploration의 강도를 조금 높임
xgboostBO.maximize(init_points=2, n_iter = 10, acq='ei', xi=0.01)

In [ ]:
params = xgboostBO.max

# 중요도 기반 변수 재선정
# 시간관계상 xgboost로 진행

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(train2_set, y_train, test_size=test_size, random_state=seed)

In [ ]:
dtrain = xgb.DMatrix(X_train,label=y_train)

In [ ]:
dval = xgb.DMatrix(X_test,label=y_test)

In [ ]:
model = xgb.train(params, dtrain)

In [ ]:
## 중요도 기반 454개
fscore = model.get_booster().get_fscore()
n_list_all = list(fscore.keys())
len(n_list_all)
new_train_set_under_454 = train2_.loc[:,n_list_all]
new_test_set_under_454 = test2_.loc[:,n_list_all]

In [ ]:
## 중요도 10이상인 249개
x_under10_del = {key: value for key, value in fscore.items() if value>10}
new_train_set_under_249 = train2_.loc[:,n_list]
new_test_set_under_249 = test2_.loc[:,n_list]

# 중요도 기반 454 변수 재학습

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(new_train_set_under_454, Y, test_size=test_size, random_state=seed)

In [ ]:
dtrain = xgb.DMatrix(X_train,label=y_train)
dval = xgb.DMatrix(X_test,label=y_test)
model = xgb.train(params, dtrain)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# 중요도 기반 249 변수 재학습

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(new_train_set_under_249, Y, test_size=test_size, random_state=seed)

In [ ]:
dtrain = xgb.DMatrix(X_train,label=y_train)
dval = xgb.DMatrix(X_test,label=y_test)
model = xgb.train(params, dtrain)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# 중요도 기반 454 변수 lgbm교차검증

In [ ]:
print(train_columns)
print(lgb_param_dart)
x_train = new_train_set_under_454.copy()
y_train = Y
x_test = new_test_set_under_454.copy()

num_class = 3
oof_train = np.zeros((len(x_train),num_class))
oof_test = np.zeros((len(x_test),num_class))
log_loss_score_list= []   
NFOLD = 5
SEED = 42

# stratifiedkfold 5 fold 사용
folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=42)
for fold_, (trn_index, val_index) in enumerate(folds.split(x_train, y_train)):
    print(f"{fold_+1} FOLD Start!!")
    trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
    val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]
    dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
    dcross = lgbm.Dataset(val_x, label=val_y, silent=True)
    
    # dart는 얼리스탑핑이 안되서 한번 num_boost_round를 넉넉히 돌린다음에 5fold에서 가장 좋았던 round로 고정하고 돌린다.
    clf = lgbm.train(lgb_param_dart, train_set=dtrain, num_boost_round=1000, valid_sets=[dtrain, dcross], 
                       verbose_eval=100)
    
    val_pred = clf.predict(val_x)
    oof_train[val_index, :] = val_pred
    
    log_loss_score = log_loss(val_y, val_pred)
    log_loss_score_list.append(log_loss_score)
    print(f"{fold_+1} FOLD LogLoss: ", log_loss_score)
    
    # 5fold 평균으로 제출
    oof_test += clf.predict(x_test[train_columns])/NFOLD

In [ ]:
reseult_test =[]

for i in oof_test:
    reseult_test.append(np.argmax(i))

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
submission = pd.DataFrame(data=reseult_test, index=sample_submission['id'])
submission.to_csv('./submission_454_window.csv', index=True)

# 중요도 기반 249 변수 lgbm 교차검증

In [ ]:
print(train_columns)
print(lgb_param_dart)
x_train = new_train_set_under_249.copy()
y_train = Y
x_test = new_test_set_under_249.copy()

num_class = 3
oof_train = np.zeros((len(x_train),num_class))
oof_test = np.zeros((len(x_test),num_class))
log_loss_score_list= []   
NFOLD = 5
SEED = 42

# stratifiedkfold 5 fold 사용
folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=42)
for fold_, (trn_index, val_index) in enumerate(folds.split(x_train, y_train)):
    print(f"{fold_+1} FOLD Start!!")
    trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
    val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]
    dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
    dcross = lgbm.Dataset(val_x, label=val_y, silent=True)
    
    # dart는 얼리스탑핑이 안되서 한번 num_boost_round를 넉넉히 돌린다음에 5fold에서 가장 좋았던 round로 고정하고 돌린다.
    clf = lgbm.train(lgb_param_dart, train_set=dtrain, num_boost_round=1000, valid_sets=[dtrain, dcross], 
                       verbose_eval=100)
    
    val_pred = clf.predict(val_x)
    oof_train[val_index, :] = val_pred
    
    log_loss_score = log_loss(val_y, val_pred)
    log_loss_score_list.append(log_loss_score)
    print(f"{fold_+1} FOLD LogLoss: ", log_loss_score)
    
    # 5fold 평균으로 제출
    oof_test += clf.predict(x_test[train_columns])/NFOLD

In [ ]:
reseult_test =[]

for i in oof_test:
    reseult_test.append(np.argmax(i))

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
submission = pd.DataFrame(data=reseult_test, index=sample_submission['id'])
submission.to_csv('./submission_249_window.csv', index=True)